In [3]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from datetime import datetime, UTC

from langchain import PromptTemplate
from langchain_ollama import ChatOllama

from app.chroma_client import get_embeddings, get_client

In [4]:
channel_name = 'go_to_vilnius'
channel_id = -1133953167

In [5]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{channel_name}', embeddings)

In [6]:
llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [7]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения о Литве и Вильнюсе в чате."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
# user_query = "Как получить детские деньги или компенсацию по уходу за ребенком."

In [8]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(user_query, k=15)
related_docs

[(Document(id='44534fd6-70e5-4560-b649-1934f1bf0f89', metadata={'date': 1674231308.0, 'id': 80006, 'sender_name': 't3ch_cat'}, page_content='похоже, пора заводить чат про кофе в вильнюсе)'),
  0.8278342120773035),
 (Document(id='d40b02d1-f166-4abf-b01c-934a6e6caf5f', metadata={'date': 1677507464.0, 'id': 82319, 'sender_name': 'orehby'}, page_content='ща бы доверять почте между РБ и Литвой'),
  0.8180362077634304),
 (Document(id='a5e66522-3275-4356-aa99-be8e4d2b980e', metadata={'date': 1724089128.0, 'id': 108527, 'sender_name': ''}, page_content='Привет, поделитесь пожалуйста контактами, если есть проверенный детский логопед в Вильнюсе.'),
  0.8169457634253323),
 (Document(id='40a1dad8-8816-46b0-a9a1-721188990962', metadata={'date': 1741357230.0, 'id': 113349, 'sender_name': 'dm22b'}, page_content='Еще можно про ВГ квартиры в Вильнюсе вбросить...'),
  0.8153703774482678),
 (Document(id='abe476d6-7788-4d2a-87e4-c875dee24c2b', metadata={'date': 1741009119.0, 'id': 113240, 'sender_name': '

In [40]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{datetime.fromtimestamp(doc.metadata['date'], UTC)} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))


2024-08-23 14:22:40+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D108686 - Коллеги, а кто нибудь пробовал с бта велнеса компенсить отель со спа? Прокатывает?


---


2024-08-23 14:25:55+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D108687 - >> Коллеги, а кто нибудь пробовал с бта велнеса компенсить отель со спа? Прокатывает?

 не знаю, как отель со спа, но баню компенсил, прокатывало)


---


2025-03-13 13:58:29+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D113452 - >> Ребята, привет!  А кто то компенсил в БТА Membership? На спа/басик? Разовые то точно компенсят, а если я возьму мембер на 5 походов, закомпенсят?

 Мне абонемент в бассейн компенсировали полностью


---


2024-05-10 14:33:38+00:00 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D105391 - А какая сейчас обстакановка со всякими ма

In [41]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{datetime.fromtimestamp(doc.metadata['date'], UTC)} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [42]:
ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)

<think>
Хорошо, пользователь спрашивает, можно ли компенсировать отель или аквапарк через велнес/БТА. Нужно проанализировать контекст, чтобы ответить.

Сначала посмотрю на сообщения. В первом вопросе коллеги спрашивают про компенсацию отеля со спа через БТА Велнес. В ответе говорится, что не знают про отель, но бани компенсировали. Затем в другом сообщении упоминается, что абонемент в бассейн компенсировали, а абонемент на 5 походов частично, если контракт закончился. Также есть упоминание, что массаж компенсировали из реабилитации, но нужно писать отдельно в БТА. В одном случае упоминается, что в East Island SPA принимают оплату по БТА, и можно компенсировать. Также есть пример с отелем-аквапарком, где спрашивают, можно ли компенсировать 160 евро. 

Видно, что в некоторых случаях компенсация возможна, особенно если услуги относятся к спа, массаж, бассейн, аквапарк. Но важно уточнить, какие именно услуги и условия. В некоторых случаях требуют дополнительные документы или коды, как в пр